In [1]:
import re
import os
import pandas as pd
import numpy as np
import traceback2
from datetime import datetime
import pymysql
import numpy as np

In [2]:
scrape_date = datetime(2021, 8,23)

In [3]:
sm = {
    "WBTC" : "BTC",
    "WETH" : "ETH",
    "WBNB" : "BNB",
    "WFTM" : "FTM",
    "WAVAX" : "AVAX",
    "cvxCRV" : "CRV",
'WAVAX' : 'AVAX', 'WETH.e' : 'ETH', 'WBTC' : 'BTC', 'USDT.e' : 'USDT', 'WBTC.e' : 'BTC', 'DAI.e' : 'DAI', 'LINK.e' : 'LINK', 'SUSHI.e' : 'SUSHI', 'AAVE.e' : 'AAVE', 'UNI.e' : 'UNI', 'YFI.e' : 'YFI', 'aaBLOCK' : 'BLOCK',
    'WETH' : 'ETH', 'WMATIC' : 'MATIC', 'WBTC' : 'BTC', 'renDOGE' : 'DOGE'
}


In [4]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)


def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [5]:
def get_raydium_data(date):
    sql = f'select * from raydium_farm_pool_data where timestamp_dt = FROM_UNIXTIME({date});'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate']
    data['network'] = 'SOL'
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

def get_liquidityfolio_pool_data(date):
    sql = f'select * from liquidityfolio_pool where timestamp_dt = FROM_UNIXTIME({date}) and `where` != "balancer";'
    data = pd.read_sql_query(sql, __connect__())
    data['apr'] = data['pool_rate'] * 100 * 12
    data['tvl'] = data['tvl'] * 1000
    data['type_of_agriculture'] = data['farm_payment'].apply(lambda x : 'FARM' if x is not None else 'POOL')
    data['network'] = 'ETH'
    return data[['pair1', 'pair2', 'apr', 'tvl', 'where', 'network', 'type_of_agriculture']]

In [6]:
date = datetime(2021, 8, 23)
dt = date.timestamp()
df = get_raydium_data(dt)
df = df.append(get_liquidityfolio_pool_data(dt))
df[df['where'] == 'Sushi']

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ETH,BEL,1.152505,8.972410e+03,Sushi,ETH,POOL
1,ETH,UMA,16.127541,2.278533e+06,Sushi,ETH,POOL
2,ETH,UWL,15.204083,6.168533e+03,Sushi,ETH,POOL
4,ETH,LIGHT,0.389757,1.884629e+03,Sushi,ETH,POOL
9,DGCL,ACXT,2.458276,8.994108e+03,Sushi,ETH,POOL
...,...,...,...,...,...,...,...
1569,USDP,DUCK,3.917236,7.109429e+04,Sushi,ETH,POOL
1570,AMIS,USDC,0.310123,4.211025e+01,Sushi,ETH,POOL
1571,ETH,SAK3,2805.896925,3.270257e+03,Sushi,ETH,POOL
1576,ETH,SEC,3.493653,8.055148e+04,Sushi,ETH,POOL


In [7]:
df

,pair1,pair2,apr,tvl,where,network,type_of_agriculture
0,ALEPH,RAY,4.540000,1.479079e+06,Raydium,SOL,POOL
1,ALEPH,USDC,6.560000,8.319639e+06,Raydium,SOL,POOL
2,APEX,USDC,21.720000,2.299912e+04,Raydium,SOL,POOL
3,APYS,USDC,3.200000,6.893120e+04,Raydium,SOL,POOL
4,AUSS,USDC,2.550000,1.768611e+02,Raydium,SOL,POOL
...,...,...,...,...,...,...,...
1573,ETH,0xMR,1.754724,4.843613e+03,Uniswap,ETH,POOL
1574,ETH,CHZ,26.608493,2.749227e+04,Uniswap,ETH,POOL
1575,ETH,POLS,17.513221,4.446122e+06,Uniswap,ETH,POOL
1576,ETH,SEC,3.493653,8.055148e+04,Sushi,ETH,POOL


In [8]:
import os
import re
import pandas as pd

In [9]:
def matched(regex, line):
    matched = re.search(regex, line)
    return bool(matched)

In [10]:
def get_info(block):
    pass

In [11]:
def clean_block(block):
    regexs = ['[\d]+ [\-]$', 'Total [s,S]taked:', 'NOTE: Once rewards', 'Each time you', "Staking Contract", "Found ", "Showing incentivized"
             ,"Finished reading smart", "The following farms have", "------------------", "{"]
    if len(block) > 0:
        for regex in regexs:
            if matched(regex, block[0]):
                return block[1:]
    return block
    

In [12]:
def get_fee(block):
    fee_regexs = ["fee [\d.]+%", "Fee [\d.]+%", "[\d.]+% fee", "[\d.]+% Fee"]
    fee_regex = "((fee|Fee) [\d.]+%)|([\d.]+% (Fee|fee))"
    for line in block:
        if matched(fee_regex, line):
            for regex in fee_regexs:
                if matched(regex, line):
                    x = re.findall(regex, line)
                    return x[0].replace("Fee", "").replace("fee", "").replace("%", "").replace(" ","").strip()
    return 0

In [13]:
block = ['[WAVAX]-[PEFI] PGL', '[+]', '[-]', '[<=>]', 'Price: $5.98 TVL: $941,883.25', 'WAVAX Price: $16.55', 'PEFI Price: $0.36', 'Staked: 143314.2957 PGL ($857,165.68)', 'PEFI Per Week: 17695.11 ($6,290.27)', 'APR: Day 0.10% Week 0.73% Year 38.16%', 'You are staking 0.00 [WAVAX]-[PEFI] PGL ($0.00), 0.00% of the pool.', 'Stake 0.00 [WAVAX]-[PEFI] PGL', 'Unstake 0.00 [WAVAX]-[PEFI] PGL - Fee 2.56%', 'Claim 0.00 PEFI ($0.00)', 'Staking or unstaking also claims rewards.']
# block = ["qweqwerlkqer", "afa 23.5% Fee adsfas"]
get_fee(block)

'2.56'

In [14]:
def get_single_pair1(line):
    if " " not in line:
        return [line, ""]

def get_single_pair2(line):
    if matched("[\d]+ [\-] ", line):
        return [line.split(" ")[2], ""]
    return ["", ""]

In [15]:
def get_pair2(line):
    if "-" in line:
        return line.split("-")
    return ["", ""]

In [16]:
def pair1(line):
#     print(line)
    try:
        is_3_pairs = line.split("]-[")
        if len(is_3_pairs) > 2:
            return ""
        s_index = line.index("[") + 1
        s = line[s_index: line.index("]", s_index)]
        return s
#         if all(ord(c) < 128 for c in s):
#             return s
#         else:
#             print(line)
#             return None
    except:
        return ''

def pair2(line):
#     print(line)
    if "-[" not in line:
        return ""
    s_index = line.index("-[") + 2
    s = line[s_index: line.index("]", s_index)]
    return s
#     print(s)
#     if all(ord(c) < 128 for c in s):
#         return s
#     else:
#         print(line)
#         return ''

In [17]:
def tvl(line):
    tvl = "TVL: \$[0-9\.\,]+"
    x = re.findall(tvl, line)
    return x[0].replace("TVL: $", "").replace("%", "").replace(",","").strip()

def get_single_tvl(line):
    exp = "\(*[0-9\,\.$]+\)*"
    x = re.findall(exp, line)
    return x[-1].replace("(", "").replace(")", "").replace("$", "").replace(",","").strip()

def where(line):
    if 'INFO' in line:
        return line.replace("INFO", "").replace('\n', '').replace(":", "").replace("//", "://").replace('*', '').strip()
    return None

def apr(line):
    apr = "Year [0-9\,\.]+\%"
    x = re.findall(apr, line)
    return x[0].replace("Year ", "").replace("%", "").strip() if len(x) else 0

In [18]:
def get_pair(block):
    return [pair1(block[0]) , pair2(block[0])]

def get_apr(block):
    for line in block:
        if matched("APR *:", line):
            return apr(line)
    return 0

def get_tvl(block, network=None):
    if network != 'BSC':
        for line in block:
            if "TVL: " in line:
                return tvl(line)
    for line in block:
        if "Staked:" in line:
            return get_single_tvl(line)
    return None

        
def get_where(block):
    return where(block[0])

In [19]:
def get_pair_info(block, where, network=None):
    if len(block)> 0:
        if matched('[\d]+ [\-]$', block[0]):
            if len(block) > 1:
                print(len(block), "\t", block[1], "Z#####################")
            else:
                print(len(block), "\t", block[0], "\t-------------------------")
        else:
            print(len(block), "\t", block[0])
    pair = get_pair(block)
    if pair is None or len(pair[0]) == 0:
        pair = get_pair(block[0])
    if pair is None or len(pair[0]) == 0:
        pair = get_single_pair1(block[0])
    if pair is None or len(pair[0]) == 0:
        pair = get_single_pair2(block[0])
    return [pair[0].replace("τ", "T", -1), pair[1].replace("τ", "T", -1), get_apr(block), get_tvl(block, network=network), where, get_fee(block)]


In [20]:
def get_pair_from_blocks(blocks, network=None):
    where = None
    pairs = []
    for block in blocks:
        block2 = block
        correction_loop = 0
        while correction_loop < 10:
            block = clean_block(block)
            correction_loop = correction_loop + 1
        if len(block) == 1:
            where_tmp = get_where(block)
            where = where_tmp
#             print(where)
        elif len(block) > 0 and len(block) < 25:
            pair = get_pair_info(block, where, network=network)
#             print("pair: ", pair)
            if pair is None or len(pair[0]) == 0:
                print('No Pair')
            else:
                pairs.append(pair)
        else:
            if(len(block)) > 0:
                print("Bigger Block as Test")
    return pairs
        

In [21]:
def convert_to_pair_blocks2(data_lines):
#     print(data_lines)
    blocks = []
    block = []
    index = 0
    while index < len(data_lines):
        line = data_lines[index]
        if "[%]" in line :
            tmp_block = block[:-3]
            if len(tmp_block):
                blocks.append(tmp_block)
            block = []
            block.append(data_lines[index - 2])
            block.append(data_lines[index - 1])
            block.append(line)
        elif "Exit" in line:
            tmp_block = block[:]
            if len(tmp_block):
                tmp_block.append(line)
                blocks.append(tmp_block)
            block = []
        elif "INFO" in line:
            blocks.append([line.strip()])
            block = []
        elif matched("[\d]+ [\-] *", line):
            blocks.append(block)
            block = []
        elif "Staking or unstaking also" in line:
            block.append(line)
            blocks.append(block)
            block = []
        else:
            block.append(line)
        index = index + 1
    if len(block):
        blocks.append(block)
#     print(blocks)
    return blocks
        

In [22]:
def extract_single_pairs(blocks):
    all_blocks = []
    for block in blocks:
        all_blocks.extend(convert_to_pair_blocks2(block))
    return all_blocks

In [23]:
def convert_to_pair_blocks(data_lines):
#     print(data_lines)
    blocks = []
    block = []
    index = 0
    while index < len(data_lines):
        line = data_lines[index]
        if "INFO" in line:
            blocks.append([line])
            block = []
        elif "[+]" in line:
            tmp_block = block[:-1]
            if len(tmp_block):
                blocks.append(tmp_block)
            block = []
            block.append(data_lines[index - 1])
            block.append(line)
        else:
            block.append(line)
            
        index = index + 1
    blocks.append(block)
    return extract_single_pairs(blocks)
        

In [24]:
def read_data_rows_from_files(file):
    f = open(file)
    lines = f.readlines()
    start_index = 0
    data_lines = []
    while start_index < len(lines):
        line = lines[start_index]
        start_index = start_index + 1
        if "INFO" in line:
#             print(line.strip())
            data_lines.append(line)
        if "Reading smart contracts" in line:
            while start_index < len(lines):
                line = lines[start_index].strip()
                start_index = start_index + 1
                if len(line):
#                     print(line)
                    data_lines.append(line)
    return convert_to_pair_blocks(data_lines)
                    

In [25]:
def convert_file_to_blocks(file_name):
    if ".DS_Store" not in file_name:
#         print(file_name)
        file_blocks = read_data_rows_from_files(file_name)
        return file_blocks
    return []

In [26]:
def conver_files_to_one_df(data_dir, network=None):
    files = os.listdir(data_dir)
    dfs = pd.DataFrame([])
    for file_name in files:
        file = f"{data_dir}/{file_name}"
#         print(file)
#         print(dfs.shape)
        if os.path.isdir(file):
            network = file_name.upper()
            if "ALL" == network:
                network = 'ETH'
            df = conver_files_to_one_df(file, network)
#             print("Dir:", network, "Total Pairs: ", df.shape)
            dfs = dfs.append(df)
#             print("Dir Return, Dfs: ", dfs.shape)
        else:
            file_blocks = convert_file_to_blocks(file)
            pairs = get_pair_from_blocks(file_blocks, network=network)
            df = pd.DataFrame(pairs, columns=['pair1', 'pair2', 'apr', 'tvl', 'where', 'fee'])
            df['network'] = network
#             print(df.shape)
            dfs = dfs.append(df)
#             print("After File Apend, Dfs: ", dfs.shape)
    return dfs.reset_index(drop=True)

In [27]:
data_dir = './out'
pair_df = conver_files_to_one_df(data_dir)
pair_df['pair1'] = pair_df['pair1'].fillna('')
pair_df['pair2'] = pair_df['pair2'].fillna('')

17 	 [MAHA]-[WETH] Uni LP
17 	 [ARTH]-[DAI] Uni LP
13 	 ARTH
8 	 Boardroom
8 	 Boardroom
8 	 Boardroom
Bigger Block as Test
17 	 [TORN]-[WETH] Uni LP
15 	 [DAI]-[DEBASE] Uni LP
11 	 DAI
15 	 [DAI]-[DEBASE] Uni LP
15 	 [DAI]-[DEBASE] Uni LP
Bigger Block as Test
12 	 [MARK 80%]-[WETH 20%]
12 	 [MARK 80%]-[USDC 20%]
15 	 [MARK]-[WETH] Uni LP
15 	 [MARK]-[USDC] Uni LP
12 	 alUSD
12 	 ALCX
16 	 [WETH]-[ALCX] SLP
11 	 alUSD3CRV-f
11 	 POOL
11 	 saddlealETH
16 	 [WETH]-[ALCX] SLP
16 	 [PLAY]-[DOUGH] SLP
16 	 [DOUGH]-[WETH] SLP
13 	 [WETH 20%]-[DOUGH 80%]
13 	 [DEFI+S 70%]-[WETH 30%]
13 	 [DEFI+L 70%]-[WETH 30%]
12 	 BCP
14 	 "reason": "cannot estimate gas; transaction may fail or may require manual gas limit",
No Pair
17 	 [LEV]-[WETH] SLP
13 	 LEV
14 	 Current Epoch: 297
No Pair
Bigger Block as Test
15 	 [WETH]-[ICE] Uni LP
15 	 [WETH]-[ICE] SLP
11 	 ICE
15 	 [WETH]-[ICE] Uni LP
8 	 Staked: 73.7780 yUSDT ($81.21)
No Pair
10 	 yvUSDT
11 	 ICE
11 	 USDT
14 	 Current Epoch: 198
No Pair
Bigger B

8 	 [uDOKI]-[UNIC] Unic Swap LP
3 	 XUNICPROXY2
8 	 [uAAH]-[WETH] Unic Swap LP
17 	 [FXS]-[WETH] Uni LP
17 	 [FRAX]-[WETH] Uni LP
17 	 [FRAX]-[USDC] Uni LP
17 	 [FXS]-[FRAX] Uni LP
14 	 FRAX3CRV-f
15 	 [SHIB]-[WETH] SLP
15 	 [LEASH]-[WETH] SLP
15 	 [WBTC]-[WETH] SLP
15 	 [USDC]-[WETH] SLP
15 	 [SUSHI]-[WETH] SLP
15 	 [UNI]-[WETH] SLP
15 	 [LINK]-[WETH] SLP
15 	 [DAI]-[WETH] SLP
15 	 [xFUND]-[WETH] SLP
15 	 [SNX]-[WETH] SLP
15 	 [WETH]-[MEME] SLP
15 	 [WETH]-[GRT] SLP
15 	 [DUCK]-[WETH] SLP
15 	 [ELON]-[WETH] SLP
15 	 [VXV]-[WETH] SLP
15 	 [BONE]-[WETH] SLP
15 	 [LEASH]-[BONE] SLP
15 	 [WETH]-[USDT] SLP
15 	 [SHIB]-[BONE] SLP
15 	 [F9]-[WETH] SLP
15 	 [UFO]-[WETH] SLP
15 	 [STARL]-[WETH] SLP
15 	 [ZIG]-[WETH] SLP
15 	 [RYOSHI]-[WETH] SLP
15 	 [WETH]-[PERL] SLP
11 	 WHEY
15 	 [WETH]-[WHEY] SLP
15 	 [CNTR]-[WETH] Uni LP
11 	 CS-WETH
11 	 CS-DAI
11 	 CS-USDT
17 	 [WOWS]-[WETH] Uni LP
14 	 [Gelato Uniswap V3 INST/ETH LP 1]
15 	 [renDOGE]-[WETH] Uni LP
15 	 [SHIB]-[WETH] Uni LP
15 	 [AKITA]-

13 	 DAI
13 	 USDC
13 	 USDT
13 	 SUSHI
13 	 WBTC
13 	 YAM
13 	 YFI
12 	 ETH
12 	 [WETH]-[USF] Uni LP
3 	 [USDT Platform] Price: $1.00 TVL: $1,389,403.96
3 	 Your Position: 0.0000 USDT [0]
8 	 Your Liquidity: 0.0000 USDT-LP [$0.00]
3 	 Your Position: 0.0000 ETH [0]
14 	 Your Liquidity: 0.0000 ETH-LP-V2 [$0.00]
13 	 USDT-LP
13 	 ETH-LP-V2
17 	 [WETH]-[COTI] Uni LP
17 	 [WETH]-[GOVI] Uni LP
17 	 [WETH]-[COTI] SLP
17 	 [WETH]-[GOVI] SLP
17 	 [XUS]-[WETH] Uni LP
17 	 [XUSD]-[XUS] Uni LP
17 	 [XUSD]-[DAI] Uni LP
17 	 [XUSD]-[WETH] Uni LP
17 	 [XUSD]-[LINK] Uni LP
17 	 [ISLA]-[WETH] Uni LP
Bigger Block as Test
16 	 [RULER]-[WETH] SLP
16 	 [RC_WBTC_25000_DAI_2021_3_31]-[DAI] SLP
16 	 [DAI]-[RC_WETH_750_DAI_2021_3_31] SLP
16 	 [DAI]-[RC_COVER_250_DAI_2021_3_31] SLP
11 	 RC_INV3CRV-f
14 	 RC_vETH23CRV-f
14 	 RC_PICKLE3CRV-f
11 	 RC_WBTC3CRV-f
11 	 RC_WETH3CRV-f
11 	 RC_xCOVER3CRV-f
11 	 RC_xRULER3CRV-f
11 	 RC_bBADGER3CRV-f
11 	 RC_RGT3CRV-f
11 	 RC_MASK3CRV-f
11 	 RC_PUNK-B3CRV-f
14 	 RC_INV3C

10 	 rvFTMa
10 	 rvLINKa
10 	 rvUSDCa
10 	 rvWBTCb
10 	 rvWFTMb
10 	 rvWFTMc
10 	 GBONE
15 	 [WFTM]-[Shiba] Uni LP
15 	 [GBONE]-[Shiba] Uni LP
15 	 [GBONE]-[WFTM] Uni LP
15 	 [WFTM]-[SHADE] SPOOKY LP
10 	 SHADE
10 	 OPERA
15 	 [WFTM]-[OPERA] Opera Swap LP
15 	 [WFTM]-[FUSD] Opera Swap LP
15 	 [WFTM]-[BTC] Opera Swap LP
15 	 [WFTM]-[ETH] Opera Swap LP
15 	 [USDC]-[WFTM] Opera Swap LP
15 	 [WFTM]-[BNB] Opera Swap LP
15 	 [WFTM]-[LINK] Opera Swap LP
15 	 [WFTM]-[SUSHI] Opera Swap LP
15 	 [WFTM]-[YFI] Opera Swap LP
15 	 [WFTM]-[ANY] Opera Swap LP
15 	 [CRV]-[WFTM] Opera Swap LP
15 	 [WFTM]-[DAI] Opera Swap LP
15 	 [WFTM]-[AAVE] Opera Swap LP
15 	 [fUSDT]-[WFTM] Opera Swap LP
15 	 [EST]-[WFTM] SPIRIT LP
10 	 EST
17 	 [WFTM]-[Nova] SPOOKY LP
15 	 [WFTM]-[HODLCC] SPOOKY LP
15 	 [WFTM]-[beHODL] SPOOKY LP
15 	 [NIPS]-[beHODL] SPOOKY LP
15 	 [REAPER]-[beHODL] SPOOKY LP
15 	 [beHODL]-[BUTT] SPOOKY LP
15 	 [beHODL]-[Shiba] SPOOKY LP
15 	 [WFTM]-[SPOON] SPIRIT LP
10 	 ibSPOON
10 	 ibFTM
10 	 ibUSDC

15 	 [RX]-[WBNB] Uni LP
10 	 OVEN
15 	 [OVEN]-[WBNB] Uni LP
15 	 [MINTv2]-[WBNB] Cake LP
15 	 [MINTv2]-[BUSD] Cake LP
15 	 [WBNB]-[BUSD] Cake LP
11 	 MINTv2
15 	 [Cake]-[BUSD] Cake LP
15 	 [WBNB]-[BUSD] Cake LP
15 	 [SUGAR]-[BUSD] Cake LP
11 	 BUSD
11 	 Cake
11 	 BUSD
11 	 WBNB
15 	 [MINTv2]-[SUGAR] Cake LP
15 	 [SUGAR]-[WBNB] Cake LP
15 	 [SUGAR]-[BUSD] Cake LP
15 	 [WBNB]-[BUSD] Cake LP
11 	 SUGAR
11 	 Cake
11 	 WBNB
11 	 BUSD
15 	 [Cake]-[WBNB] Cake LP
11 	 BTCB
11 	 ETH
15 	 [MINTv2]-[SUGAR] Cake LP
15 	 [MINTv2]-[WBNB] Cake LP
15 	 [MINTv2]-[BUSD] Cake LP
11 	 MINTv2
10 	 TeaSportV1
15 	 [WBNB]-[TeaSportV1] Cake LP
15 	 [BUSD]-[TeaSportV1] Cake LP
11 	 LINK
15 	 [WBNB]-[TeaSportV1] Cake LP
15 	 [BUSD]-[TeaSportV1] Cake LP
15 	 [WBNB]-[BUSD] Cake LP
15 	 [MINTv2]-[WBNB] Cake LP
15 	 [MINTv2]-[BUSD] Cake LP
15 	 [SUGAR]-[WBNB] Cake LP
15 	 [SUGAR]-[BUSD] Cake LP
15 	 [Cake]-[BUSD] Cake LP
11 	 WBNB
11 	 BUSD
11 	 TeaSportV1
15 	 [MINTv2]-[SUGAR] Cake LP
11 	 Cake
15 	 [AUTO]-[WBNB] 

...

11 	 USDC
11 	 WMATIC
11 	 USDT
11 	 WBTC
11 	 WETH
11 	 QUICK
11 	 AAVE
15 	 [USDC]-[COKE] Uni LP
15 	 [WMATIC]-[COKE] Uni LP
11 	 COKE
15 	 [WMATIC]-[USDC] Uni LP
11 	 WMATIC
11 	 QUICK
11 	 WETH
11 	 WBTC
11 	 AAVE
11 	 DAI
11 	 USDT
11 	 USDC
11 	 SUSHI
10 	 FISH
15 	 [USDC]-[KUL] Uni LP
15 	 [WMATIC]-[KUL] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[USDT] Uni LP
15 	 [USDC]-[DAI] Uni LP
15 	 [WMATIC]-[WETH] Uni LP
15 	 [WMATIC]-[WBTC] Uni LP
15 	 [WBTC]-[WETH] Uni LP
15 	 [WMATIC]-[QuickChart] Uni LP
15 	 [WMATIC]-[COMMUNITY] Uni LP
15 	 [WMATIC]-[HAN] Uni LP
15 	 [WMATIC]-[DPS] Uni LP
15 	 [WMATIC]-[MORDOR] Uni LP
15 	 [WMATIC]-[SIN] Uni LP
15 	 [WMATIC]-[HND] Uni LP
10 	 KUL
11 	 WMATIC
11 	 USDC
11 	 WBTC
11 	 WETH
11 	 USDT
11 	 DAI
15 	 [USDC]-[TULIP] Uni LP
15 	 [WMATIC]-[TULIP] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[WETH] Uni LP
15 	 [WBTC]-[USDC] Uni LP
15 	 [USDC]-[FISH] Uni LP
11 	 USDC
11 	 WMATIC
11 	 WETH
11 	 WBTC
11 	 AAVE
11 	 LINK
11 	 FISH
11 	 USDT


15 	 [USDC]-[FISH] SLP
15 	 [WMATIC]-[FISH] SLP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[FISH] Uni LP
15 	 [WMATIC]-[FISH] Uni LP
10 	 POLYTOKEN
15 	 [USDC]-[POLYTOKEN] Uni LP
15 	 [WMATIC]-[POLYTOKEN] Uni LP
15 	 [WMATIC]-[PolyFi] Uni LP
11 	 WMATIC
11 	 USDT
11 	 USDC
11 	 DAI
11 	 WBTC
11 	 WETH
11 	 QUICK
11 	 AAVE
15 	 [WMATIC]-[KOGECOIN] Uni LP
11 	 KOGECOIN
15 	 [Krill]-[USDC] Uni LP
11 	 WMATIC
11 	 WBTC
11 	 WETH
11 	 QUICK
11 	 USDT
11 	 USDC
11 	 Krill
11 	 AAVE
11 	 LINK
11 	 DAI
11 	 SUSHI
10 	 W
10 	 WEXpoly
15 	 [WMATIC]-[WEXpoly] Uni LP
15 	 [USDC]-[WEXpoly] Uni LP
15 	 [WMATIC]-[WETH] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[WETH] Uni LP
15 	 [WBTC]-[WETH] Uni LP
15 	 [WBTC]-[USDC] Uni LP
15 	 [USDC]-[USDT] Uni LP
15 	 [USDC]-[DAI] Uni LP
15 	 [WETH]-[DAI] Uni LP
15 	 [WETH]-[AAVE] Uni LP
15 	 [LINK]-[WETH] Uni LP
15 	 [SNX]-[WETH] Uni LP
15 	 [WMATIC]-[PAUTO] Uni LP
15 	 [WMATIC]-[BIFI] Uni LP
15 	 [WMATIC]-[ELE] Uni LP
15 	 [WMATIC]-[PolyDoge] Uni LP
15 	 [WMA

10 	 GMS
15 	 [WMATIC]-[USDC] SLP
15 	 [WMATIC]-[WETH] SLP
15 	 [USDC]-[WETH] SLP
15 	 [WBTC]-[WETH] SLP
15 	 [WMATIC]-[WOOFY] SLP
15 	 [SUSHI]-[WETH] SLP
15 	 [GMS]-[USDC] SLP
15 	 [WMATIC]-[GMS] SLP
10 	 kmWMATIC/USDC-LINK
10 	 kmWETH/WMATIC-LINK
10 	 kmWETH/USDT-LINK
10 	 kmAAVE/USDC-LINK
10 	 kmWBTC/DAI-LINK
15 	 [USDC]-[SDS] Uni LP
15 	 [USDC]-[sUSDC] Uni LP
15 	 [USDC]-[SDO] Uni LP
15 	 [USDC]-[WETH] Uni LP
15 	 [WMATIC]-[sMATIC] Uni LP
15 	 [SDO]-[sMATIC] Uni LP
11 	 YELD V2
No Pair
11 	 WMATIC
11 	 WBTC
11 	 WETH
11 	 USDT
11 	 USDC
11 	 DAI
11 	 PolyDoge
10 	 ICE
15 	 [WMATIC]-[YELD V2] APE LP
15 	 [YELD V2]-[USDC] APE LP
15 	 [WMATIC]-[YELD V2] SLP
11 	 LINK
10 	 BANANA
15 	 [Fairy]-[USDC] Uni LP
15 	 [Fairy]-[WMATIC] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
11 	 Fairy
11 	 WMATIC
11 	 DAI
11 	 USDC
11 	 WETH
11 	 QUICK
11 	 USDT
15 	 [USDC]-[DAI] Uni LP
15 	 [USDC]-[DUFF] Uni LP
11 	 WMATIC
11 	 DUFF
15 	 [USDC]-[WETH] Uni LP
11 	 WETH
11 	 DAI
11 	 QUICK
11 	 Krill
11 	 MUST
10 	

11 	 USDC
11 	 PolyZeus
15 	 [USDC]-[USDT] Uni LP
15 	 [WBTC]-[USDC] Uni LP
15 	 [WMATIC]-[QUICK] Uni LP
15 	 [WMATIC]-[PolyZeus] Uni LP
15 	 [USDC]-[PolyZeus] Uni LP
5 	 Rewards start at block
No Pair
15 	 [USDC]-[KAVIAN] Uni LP
15 	 [WMATIC]-[KAVIAN] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[WETH] Uni LP
15 	 [USDC]-[DAI] Uni LP
15 	 [WMATIC]-[WETH] Uni LP
15 	 [USDC]-[USDT] Uni LP
11 	 KAVIAN
11 	 WMATIC
11 	 QUICK
11 	 DAI
11 	 WETH
10 	 IRIS
15 	 [WETH]-[AAVE] Uni LP
15 	 [LINK]-[QUICK] Uni LP
15 	 [SUSHI]-[WETH] SLP
15 	 [WBTC]-[WETH] Uni LP
15 	 [USDC]-[GLOCK] Uni LP
11 	 GLOCK
11 	 WMATIC
15 	 [USDC]-[USDT] Uni LP
11 	 WETH
11 	 DAI
11 	 QUICK
11 	 WBTC
11 	 USDT
15 	 [USDC]-[RAGNAROK] Uni LP
15 	 [WMATIC]-[RAGNAROK] Uni LP
11 	 Zero
15 	 [WMATIC]-[Zero] Uni LP
11 	 USDC
11 	 WMATIC
11 	 WETH
15 	 [WMATIC]-[MAPLE] Uni LP
15 	 [WBTC]-[WETH] Uni LP
15 	 [USDC]-[ELECTRON] SLP
15 	 [WMATIC]-[ELECTRON] SLP
11 	 ELECTRON
15 	 [WMATIC]-[USDC] SLP
11 	 USDC
11 	 WMATIC
11 	 WETH


11 	 USDC
11 	 WBTC
11 	 WETH
11 	 QUICK
11 	 USDT
11 	 DAI
15 	 [USDC]-[PAN] Uni LP
15 	 [WETH]-[PAN] Uni LP
15 	 [WMATIC]-[PAN] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
15 	 [USDC]-[WETH] Uni LP
11 	 PAN
11 	 WMATIC
11 	 PolyDoge
11 	 WETH
11 	 WBTC
11 	 AAVE
11 	 QUICK
10 	 FISH
11 	 LINK
11 	 USDC
11 	 DAI
11 	 USDT
13 	 MYFRIENDS
17 	 [USDC]-[ARCADIUM] Uni LP
17 	 [WMATIC]-[ARCADIUM] Uni LP
17 	 [USDC]-[MYFRIENDS] Uni LP
17 	 [WMATIC]-[MYFRIENDS] Uni LP
17 	 [ARCADIUM]-[MYFRIENDS] Uni LP
17 	 [USDC]-[BONE] Uni LP
17 	 [WMATIC]-[USDC] Uni LP
17 	 [USDC]-[WETH] Uni LP
17 	 [USDC]-[USDT] Uni LP
17 	 [USDC]-[DAI] Uni LP
17 	 [DAI]-[USDT] Uni LP
13 	 ARCADIUM
13 	 WMATIC
13 	 QUICK
13 	 WETH
13 	 WBTC
12 	 PUP
12 	 BALL
12 	 BIFI
13 	 DAI
13 	 USDT
13 	 USDC
12 	 miMATIC
13 	 LINK
12 	 FISH
13 	 PolyDoge
15 	 [USDC]-[ABUNNY] Uni LP
15 	 [WMATIC]-[ABUNNY] Uni LP
15 	 [USDC]-[WETH] Uni LP
15 	 [WMATIC]-[USDC] Uni LP
11 	 ABUNNY
11 	 WBTC
15 	 [WBTC]-[WETH] Uni LP
11 	 AAVE
11 	 QUICK
15 	 [Fal

In [28]:
pair_df[pair_df['network']=='BSC']

,pair1,pair2,apr,tvl,where,fee,network
1506,xSAT,WBNB,180.78,280.41,https://lazymint.finance,0,BSC
1507,WBNB,BUSD,41.19,4922.59,https://lazymint.finance,0,BSC
1508,BTCB,WBNB,61.50,3297.39,https://lazymint.finance,0,BSC
1509,ETH,WBNB,266.00,571.74,https://lazymint.finance,0,BSC
1510,USDT,BUSD,215.31,470.90,https://lazymint.finance,0,BSC
...,...,...,...,...,...,...,...
10369,USDC,BUSD,85.76,102589.41,https://app.barbecueswapfinance.com,3,BSC
10370,BANANA,,270.86,32481.96,https://app.barbecueswapfinance.com,3,BSC
10371,BTCB,,124.45,70694.25,https://app.barbecueswapfinance.com,3,BSC
10372,BREW,WBNB,447.04,19680.50,https://app.barbecueswapfinance.com,3,BSC


In [29]:
pair_df[pair_df['fee'].notna()]

,pair1,pair2,apr,tvl,where,fee,network
0,MAHA,WETH,0.00,814332.98,https://www.arthcoin.com,0,ETH
1,ARTH,DAI,0.00,12754.52,https://www.arthcoin.com,0,ETH
2,ARTH,,0.00,642.97,https://www.arthcoin.com,0,ETH
3,Boardroom,,0,None,https://www.arthcoin.com,0,ETH
4,Boardroom,,0,None,https://www.arthcoin.com,0,ETH
...,...,...,...,...,...,...,...
14528,WBTC.e,WAVAX,242.51,32955.73,https://www.lydia.finance,0,AVAX
14529,USDT.e,DAI.e,34.43,60070.76,https://www.lydia.finance,0,AVAX
14530,LINK.e,WAVAX,265.83,75071.76,https://www.lydia.finance,0,AVAX
14531,AVME,WAVAX,189.97,827546.51,https://avme.io,0,AVAX


In [30]:
pair_df['apy'] = np.nan

In [31]:
def get_symbol_mapping(pair):
    if pair is not None:
        return sm[pair.strip()] if pair.strip() in sm else pair.strip()
    else:
        return ""

def update_apr(apr):
    apr = f"{apr}".replace("%", "")
    if len(apr) == 0 or "nan" == apr:
        return np.nan
    else:
        return apr

In [32]:
pair_df['type_of_agriculture'] = 'FARM'
pair_df = pair_df.append(df).reset_index(drop=True)

In [33]:
pair_df

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
0,MAHA,WETH,0.00,814332.98,https://www.arthcoin.com,0,ETH,NaN,FARM
1,ARTH,DAI,0.00,12754.52,https://www.arthcoin.com,0,ETH,NaN,FARM
2,ARTH,,0.00,642.97,https://www.arthcoin.com,0,ETH,NaN,FARM
3,Boardroom,,0,None,https://www.arthcoin.com,0,ETH,NaN,FARM
4,Boardroom,,0,None,https://www.arthcoin.com,0,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...,...
16249,ETH,0xMR,1.754724,4843.612801,Uniswap,NaN,ETH,NaN,POOL
16250,ETH,CHZ,26.608493,27492.265558,Uniswap,NaN,ETH,NaN,POOL
16251,ETH,POLS,17.513221,4446122.216257,Uniswap,NaN,ETH,NaN,POOL
16252,ETH,SEC,3.493653,80551.479431,Sushi,NaN,ETH,NaN,POOL


### scrapped Data

In [34]:
scrapped_df = pd.read_csv('scrapped_pairs.csv')
scrapped_df['fee'] = 0
scrapped_df['apy'] = np.nan
scrapped_df['type_of_agriculture'] = 'POOL'

In [35]:
scrapped_df

,tvl,apr,where,network,pair1,pair2,fee,apy,type_of_agriculture
0,187696956,11.01,Pangolin,ETH,PNG,WAVAX,0,NaN,POOL
1,50915377,94.00,Pangolin,ETH,WETH.e,WAVAX,0,NaN,POOL
2,41292652,55.20,Pangolin,ETH,QI,WAVAX,0,NaN,POOL
3,11865020,6.97,Pangolin,ETH,WAVAX,ETH,0,NaN,POOL
4,11393745,36.98,Pangolin,ETH,WAVAX,XAVA,0,NaN,POOL
...,...,...,...,...,...,...,...,...,...
365,0.00,0.00,TraderJoe,ETH,PNG,DAI,0,NaN,POOL
366,0.00,0.00,TraderJoe,ETH,DALTON,WAVAX,0,NaN,POOL
367,0.00,0.00,TraderJoe,ETH,shit,WAVAX,0,NaN,POOL
368,0.00,0.00,TraderJoe,ETH,PNG,XAVA,0,NaN,POOL


In [36]:
pair_df = pair_df.append(scrapped_df).reset_index(drop=True)

In [37]:
pair_df

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
0,MAHA,WETH,0.00,814332.98,https://www.arthcoin.com,0,ETH,NaN,FARM
1,ARTH,DAI,0.00,12754.52,https://www.arthcoin.com,0,ETH,NaN,FARM
2,ARTH,,0.00,642.97,https://www.arthcoin.com,0,ETH,NaN,FARM
3,Boardroom,,0,None,https://www.arthcoin.com,0,ETH,NaN,FARM
4,Boardroom,,0,None,https://www.arthcoin.com,0,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...,...
16619,PNG,DAI,0.0,0.00,TraderJoe,0,ETH,NaN,POOL
16620,DALTON,WAVAX,0.0,0.00,TraderJoe,0,ETH,NaN,POOL
16621,shit,WAVAX,0.0,0.00,TraderJoe,0,ETH,NaN,POOL
16622,PNG,XAVA,0.0,0.00,TraderJoe,0,ETH,NaN,POOL


Addidtional Pairs

In [38]:
ext_grades = pd.DataFrame([])
excel_file_name = 'addition_yield_scrape_8_23_formatted.xlsx'
ap_sh1_df = pd.read_excel(excel_file_name, "Additional Staking Yields")
ext_farm_grades1 = ap_sh1_df[['Website', 'network', 'Grade']]
ext_farm_grades1.columns = ['Website', 'Network', 'Grade']
ap_sh1_df = ap_sh1_df[['pair1', 'pair2', 'APR', 'APY', 'fee', 'network', 'type_of_agriculture', 'dex', 'TLV']]
ap_sh1_df.columns = ['pair1', 'pair2', 'apr', 'apy', 'fee', 'network', 'type_of_agriculture', 'where', 'tvl']
ap_sh1_df['pair1'] = ap_sh1_df['pair1'].fillna('')
ap_sh1_df['pair2'] = ap_sh1_df['pair2'].fillna('')
ap_sh1_df['apy'] = np.nan
ap_sh2_df = pd.read_excel(excel_file_name, "Visor Manual Scrub")
print(ap_sh2_df.columns)
ext_farm_grades2 = ap_sh2_df[['dex', 'network', 'Grade']]
ext_farm_grades2.columns = ['Website', 'Network', 'Grade']
ap_sh2_df = ap_sh2_df[['pair1', 'pair2', 'APR', 'fee', 'network', 'tvl', 'type_of_agriculture', 'dex']]
ap_sh2_df.columns = ['pair1', 'pair2', 'apr', 'fee', 'network', 'tvl', 'type_of_agriculture', 'where']
ap_sh2_df['apy'] = np.nan
ap_sh2_df['pair1'] = ap_sh2_df['pair1'].fillna('')
ap_sh2_df['pair2'] = ap_sh2_df['pair2'].fillna('')
ext_pair_df = ap_sh1_df.append(ap_sh2_df).reset_index(drop=True)
ext_pair_df['network'] = ext_pair_df['network'].fillna('')
ext_pair_df['apr'] = ext_pair_df['apr'] * 100
ext_grades = ext_farm_grades1.append(ext_farm_grades2).drop_duplicates().reset_index(drop=True)
# pair_df = ext_pair_df
pair_df = pair_df.append(ext_pair_df)
ext_grades

Index(['website', 'pair1', 'pair2', 'APR', 'fee', 'network', 'tvl',
       'type_of_agriculture', 'Grade', 'Date', 'dex'],
      dtype='object')


,Website,Network,Grade
0,https://88mph.app/rewards,ETH,B
1,https://account.aleph.im/#/,ETH,B
2,https://app.alpacafinance.org/stake,BSC,B
3,https://homora-bsc.alphafinance.io/earn,BSC,B
4,https://app.anchorprotocol.com/earn,Terra,B
5,https://stakefi.ankr.com/features,ETH,B
6,https://app.barnbridge.com/yield-farming,ETH,B
7,https://belt.fi/bsc,BSC,B
8,https://www.convexfinance.com/stake,ETH,B
9,https://app.cream.finance/stake,ETH,B


In [39]:
pair_df = pair_df.drop(pair_df[(pair_df['pair1'] == '') & (pair_df['pair2'] == '')].index)
pair_df[pair_df['pair1'] == '']

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
15319,,sil,0.0,0.0,Sushi,NaN,ETH,NaN,POOL
15639,,xSUSHI,0.0,0.0,Sushi,NaN,ETH,NaN,POOL


In [40]:

pair_df['apr'] = pair_df['apr'].apply(lambda x: update_apr(x))
pair_df['pair1'] = pair_df['pair1'].apply(lambda x: sm[x.strip()] if x.strip() in sm else x.strip())
pair_df['pair2'] = pair_df['pair2'].apply(lambda x: get_symbol_mapping(x))

In [41]:
def convert_apr_to_apy(pair_df):
    tmp_df = pair_df[pair_df['apy'].isna()]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apr = row['apr']
        if pd.isnull( apr):
            return
        if apr is not None and len(apr.strip()):
            apr = float(apr.replace(',',''))
        else:
            apr = 0
        try:
            pair_df.loc[index, 'apy'] = ((1+(apr/100)/365)**365-1)*100
        except:
            pair_df.loc[index, 'apy'] = np.nan
    return
            
def convert_apy_to_apr(pair_df):
    tmp_df = pair_df[(pair_df['apr'].isnull())]
    print(tmp_df.shape)
    for index, row in tmp_df.iterrows():
        apy = row['apy']
        if pd.isnull( apy) or pd.isna(apy):
            return
        apy = f"{apy}"
        if apy is not None and len(apy.strip()):
            apy = float(apy.replace(',',''))
        else:
            apy = 0
        try:
            pair_df.loc[index, 'apr'] = (((1+(apy/100))**(1/365)-1)*365)*100
        except:
            pair_df.loc[index, 'apr'] = np.nan
    return


convert_apr_to_apy(pair_df)
convert_apy_to_apr(pair_df)

(16673, 9)
(13, 9)


In [42]:
pair_df[pair_df['apy'].isna()]

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
76,Boardroom,,1708943.63,None,https://onecash.finance,0,ETH,NaN,FARM
617,TUSD,Legacy tfTUSD,241220.50,3866.89,https://app.truefi.io/farm,0,ETH,NaN,FARM
637,USDC,,1886240.38,60.06,https://app.sirenmarkets.com/stake,0,ETH,NaN,FARM
818,RULER,ETH,31652220292.44,2495079.13,https://app.rulerprotocol.com/app/farms,0,ETH,NaN,FARM
819,RC_WBTC_25000_DAI_2021_3_31,DAI,99045787200222232576.00,1449.41,https://app.rulerprotocol.com/app/farms,0,ETH,NaN,FARM
...,...,...,...,...,...,...,...,...,...
14215,BTC,ETH,7188367.53,97593069.82,https://www.astrobunny.finance,4,MATIC,NaN,FARM
14221,USDC,ETH,45776347388.70,148230913.24,https://polyfalcon.com/,4,MATIC,NaN,FARM
14283,QUICK,,15316529.48,0.00,https://www.polysnake.finance,4,MATIC,NaN,FARM
14384,BTC,AVAX,21944074330101.21,783.61,https://pandaswap.exchange,0,AVAX,NaN,FARM


In [43]:
pair_df[pair_df['pair1']=='MPH']

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
191,MPH,ETH,8.77,461721.12,https://www.bao.finance,0,ETH,9.164907,FARM
0,MPH,,6.78,7612706.48,https://88mph.app/rewards,0,ETH,7.014452,Staking


In [44]:
pair_df = pair_df.drop_duplicates()
pair_df

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
0,MAHA,ETH,0.00,814332.98,https://www.arthcoin.com,0,ETH,7.014452e+00,FARM
1,ARTH,DAI,0.0,12754.52,https://www.arthcoin.com,0,ETH,0.000000e+00,FARM
2,ARTH,,0.00,642.97,https://www.arthcoin.com,0,ETH,0.000000e+00,FARM
3,Boardroom,,0,None,https://www.arthcoin.com,0,ETH,0.000000e+00,FARM
4,Boardroom,,0.0,None,https://www.arthcoin.com,0,ETH,0.000000e+00,FARM
...,...,...,...,...,...,...,...,...,...
44,LINK,ETH,96.78999999999999,NaN,https://www.visor.finance/,0,ETH,3.197184e+114,FARM
45,BTC,ETH,14.530000000000001,NaN,https://www.visor.finance/,0,ETH,0.000000e+00,FARM
46,DAI,ETH,40.550000000000004,NaN,https://www.visor.finance/,0,ETH,0.000000e+00,FARM
47,USDC,ETH,47.910000000000004,NaN,https://www.visor.finance/,0,ETH,1.779317e+45,FARM


In [45]:
pair_df[pair_df['pair1'] == '']

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
15319,,sil,0.0,0.0,Sushi,NaN,ETH,0.0,POOL
15639,,xSUSHI,0.0,0.0,Sushi,NaN,ETH,0.0,POOL


In [46]:
single_piar_df = pair_df[pair_df['pair2'] == ""].copy()
double_coin_pair_df = pair_df[pair_df['pair2'] != ""].copy()
tmp_df = double_coin_pair_df.copy()

In [47]:
tmp_df['tmp'] = tmp_df['pair1']
tmp_df['pair1'] = tmp_df['pair2']
tmp_df['pair2'] = tmp_df['tmp']
del tmp_df['tmp']

In [48]:
double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
0,,sil,0.0,0.0,Sushi,NaN,ETH,0.000000,POOL
1,,xSUSHI,0.0,0.0,Sushi,NaN,ETH,0.000000,POOL
2,$ANRX,ETH,0.01,190537,QuickSwap,0,ETH,0.010000,POOL
3,$BASED,ETH,21.71612232,7955.295761,Uniswap,NaN,ETH,24.246418,POOL
4,$BASED,ETH,0.016873199999999998,0.592761,Sushi,NaN,ETH,0.016875,POOL
...,...,...,...,...,...,...,...,...,...
21245,🐟,USDC,0.00,10166.36,https://penguinswap.eth.link,0,ETH,0.000000,FARM
21246,🐟,⛩️,0.00,21413.38,https://penguinswap.eth.link,0,ETH,0.000000,FARM
21247,💎,AVAX,8.05,46875,Pangolin,0,ETH,8.381923,POOL
21248,🚜,MATIC,34.78,5109.78,https://www.honestwork.farm/,0,MATIC,41.571457,FARM


In [49]:
summary = {
    "total_pairs": pair_df.shape[0],
"single_coin_pairs": single_piar_df.shape[0],
"double_coin_piars": double_coin_pair_df.shape[0],
"after_switching_coins_total_double_coin_pairs": double_coin_pair_df.shape[0] * 2,
"total_pairs_in_sheet": double_coin_pair_df.shape[0] * 2 + single_piar_df.shape[0]
}

In [50]:
summary

{'total_pairs': 16566,
 'single_coin_pairs': 5941,
 'double_coin_piars': 10625,
 'after_switching_coins_total_double_coin_pairs': 21250,
 'total_pairs_in_sheet': 27191}

In [51]:
pair_df = double_coin_pair_df.append(tmp_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = single_piar_df.append(pair_df).sort_values(['pair1', 'pair2']).reset_index(drop=True)
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == ''].index)
pair_df = pair_df.drop(pair_df[pair_df['pair1'] == '$0.00'].index)

In [52]:
pair_df

,pair1,pair2,apr,tvl,where,fee,network,apy,type_of_agriculture
4,$ANRX,ETH,0.01,190537,QuickSwap,0,ETH,0.010000,POOL
5,$BASED,ETH,21.71612232,7955.295761,Uniswap,NaN,ETH,24.246418,POOL
6,$BASED,ETH,0.016873199999999998,0.592761,Sushi,NaN,ETH,0.016875,POOL
7,$BASED,SUSD,21.43906008,6866.370343,Uniswap,NaN,ETH,23.902857,POOL
8,$DG,BNB,205.57,3092860.44,https://pancakeswap.finance,0,BSC,676.737863,FARM
...,...,...,...,...,...,...,...,...,...
27186,🐟,⛩️,0.00,21413.38,https://penguinswap.eth.link,0,ETH,0.000000,FARM
27187,💎,AVAX,8.05,46875,Pangolin,0,ETH,8.381923,POOL
27188,🚜,,129.80,2188.99,https://www.honestwork.farm/,0,MATIC,265.354346,FARM
27189,🚜,MATIC,34.78,5109.78,https://www.honestwork.farm/,0,MATIC,41.571457,FARM


In [53]:
pair_df['dex'] = pair_df['where']
pair_df['date'] = scrape_date 
del pair_df['where']

In [54]:
pair_df['dex'].unique().tolist()

['QuickSwap',
 'Uniswap',
 'Sushi',
 'https://pancakeswap.finance',
 'https://quickswap.exchange/#/quick',
 'https://app.sushi.com',
 'https://dinoswap.exchange',
 'https://strudel.finance',
 'https://swap.cometh.io/#/stake',
 'https://cvi.finance',
 'https://blazedefi.finance',
 'https://www.bao.finance',
 'https://caramelswap.finance',
 'https://poopswap.finance',
 'https://absorber.finance',
 'https://brussels.worldswap.finance',
 'https://icecreamswap.finance',
 'http://mangofarm.finance',
 'https://berlin.worldswap.finance',
 'https://snakedefi.com',
 'https://cobra.exchange',
 'https://www.convexfinance.com',
 'https://ellipsis.finance',
 'https://app.swamp.finance',
 'https://app.firebird.finance',
 'https://frozenyogurt.finance',
 'https://nerve.fi',
 'https://beta.belt.fi',
 'https://autofarm.network',
 'https://ten.finance',
 'https://app.yieldparrot.finance',
 'https://blue.planetfinance.io',
 'https://cashcowswap.org',
 '',
 'https://glacierswap.org/farms/',
 'https://8ball

In [55]:
pair_df[pair_df['pair1'] == 'ETH']

,pair1,pair2,apr,tvl,fee,network,apy,type_of_agriculture,dex,date
11807,ETH,,0.00,334.66,0,ETH,0.000000e+00,FARM,https://app.thisistheway.finance/#/stake,2021-08-23
11808,ETH,,0,7636.83,0,ETH,0.000000e+00,FARM,https://www.bigdataprotocol.com/datavault,2021-08-23
11809,ETH,,0.00,128775.91,0,ETH,0.000000e+00,FARM,None,2021-08-23
11810,ETH,,0.00,13633833.57,0,ETH,0.000000e+00,FARM,https://app.cryptex.finance,2021-08-23
11811,ETH,,46466.72,4.95,4,FANTOM,1.455579e+132,FARM,https://rediant.finance,2021-08-23
...,...,...,...,...,...,...,...,...,...,...
14831,ETH,yyDAI+yUSDC+yUSDT+yTUSD,1.15,323638.55,0,ETH,1.156620e+00,FARM,https://www.bao.finance,2021-08-23
14832,ETH,zHEGIC,3.2869177200000004,2077.65917,NaN,ETH,3.341381e+00,POOL,Uniswap,2021-08-23
14833,ETH,zLOT,2.7645146400000002,2378.355245,NaN,ETH,2.802974e+00,POOL,Sushi,2021-08-23
14834,ETH,🌾,184.86,3218.98,4,MATIC,5.321361e+02,FARM,https://harvester.app,2021-08-23


In [56]:
farm_grades = pd.read_csv('Yield_grades.csv')
farm_grades

,Farm,Website,Network,RugDoc / Indv.,Grade
0,Aave,https://aave.com,ETH,NaN,A
1,Armor,https://armor.fi,ETH,NaN,A
2,UniSwap (Pool),uniswap.org,ETH,NaN,A
3,Aave,https://aave.com,MATIC,NaN,A
4,Sushi,https://app.sushi.com,MATIC,NaN,A
...,...,...,...,...,...
765,NaN,https://app.pendle.finance/,ETH,NaN,D
766,NaN,https://shibaswap.com,ETH,NaN,B
767,NaN,Pangolin,ETH,NaN,B
768,NaN,QuickSwap,ETH,NaN,B


In [57]:
farm_grades = farm_grades[['Website', 'Network', 'Grade']]
farm_grades = farm_grades.append(ext_grades).reset_index(drop=True)
farm_grades = farm_grades.append({'Website': 'Sushi', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'https://app.sushi.com/farms/', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)
farm_grades = farm_grades.append({'Website': 'Uniswap', 'Network': 'ETH', 'Grade': 'A'}, ignore_index=True)

In [58]:
pairs_farm_risk_df = pd.merge(pair_df, farm_grades, how='left', left_on=['dex'], right_on=['Website'])
pairs_farm_risk_df.to_csv(f'inhouse_farm_risk_yeild_report_{date.day}-{date.month}-{date.year}.csv')
pairs_farm_risk_df


,pair1,pair2,apr,tvl,fee,network,apy,type_of_agriculture,dex,date,Website,Network,Grade
0,$ANRX,ETH,0.01,190537,0,ETH,0.010000,POOL,QuickSwap,2021-08-23,QuickSwap,ETH,B
1,$BASED,ETH,21.71612232,7955.295761,NaN,ETH,24.246418,POOL,Uniswap,2021-08-23,Uniswap,ETH,A
2,$BASED,ETH,0.016873199999999998,0.592761,NaN,ETH,0.016875,POOL,Sushi,2021-08-23,Sushi,ETH,A
3,$BASED,SUSD,21.43906008,6866.370343,NaN,ETH,23.902857,POOL,Uniswap,2021-08-23,Uniswap,ETH,A
4,$DG,BNB,205.57,3092860.44,0,BSC,676.737863,FARM,https://pancakeswap.finance,2021-08-23,https://pancakeswap.finance,BSC,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28895,🐟,⛩️,0.00,21413.38,0,ETH,0.000000,FARM,https://penguinswap.eth.link,2021-08-23,https://penguinswap.eth.link,ETH,D
28896,💎,AVAX,8.05,46875,0,ETH,8.381923,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
28897,🚜,,129.80,2188.99,0,MATIC,265.354346,FARM,https://www.honestwork.farm/,2021-08-23,https://www.honestwork.farm/,MATIC,D
28898,🚜,MATIC,34.78,5109.78,0,MATIC,41.571457,FARM,https://www.honestwork.farm/,2021-08-23,https://www.honestwork.farm/,MATIC,D


In [59]:
pairs_farm_risk_df[(pairs_farm_risk_df['Website'].isna()) & (pairs_farm_risk_df['type_of_agriculture'] == 'POOL')]

,pair1,pair2,apr,tvl,fee,network,apy,type_of_agriculture,dex,date,Website,Network,Grade


In [60]:
unmapped_dexs_df = pairs_farm_risk_df[pairs_farm_risk_df['Website'].isna()][['dex', 'network']]
unmapped_dexs_df = unmapped_dexs_df.drop_duplicates().reset_index(drop=True)
if unmapped_dexs_df.shape[0] > 1:
    unmapped_dexs_df.to_csv('unmapped_dex.csv')
unmapped_dexs_df


,dex,network
0,https://dinoswap.exchange,MATIC
1,https://cobra.exchange,BSC
2,,BSC
3,https://www.uniron.finance,FANTOM
4,https://www.knight-finance.xyz,FANTOM
...,...,...
276,https://polygon-triton.galaxyfinance.one/,MATIC
277,https://app.blackswan.network,MATIC
278,https://mlnl.finance | Vaults at https://milfi...,FANTOM
279,https://snowball.network/earn/,AVAX


In [61]:
pairs_farm_risk_df[pairs_farm_risk_df['pair1'] == '']

,pair1,pair2,apr,tvl,fee,network,apy,type_of_agriculture,dex,date,Website,Network,Grade


In [62]:
pairs_farm_risk_df[pairs_farm_risk_df['dex'] =='Pangolin']

,pair1,pair2,apr,tvl,fee,network,apy,type_of_agriculture,dex,date,Website,Network,Grade
192,AAVE,AVAX,35.19,214058,0,ETH,42.152533,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
193,AAVE,AVAX,14.85,54773,0,ETH,16.005776,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
230,AAVE,PNG,36.83,282249,0,ETH,44.500721,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
231,AAVE,PNG,6.25,20587,0,ETH,6.448876,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
555,AKBASH,AVAX,32.02,26484,0,ETH,37.720989,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...
27644,YFI,AVAX,10.06,110251,0,ETH,10.581889,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
27645,YFI,AVAX,1.45,13047,0,ETH,1.460534,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
28886,♀,AVAX,5.11,23419,0,ETH,5.242437,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
28890,🐆,AVAX,13.59,46852,0,ETH,14.553836,POOL,Pangolin,2021-08-23,Pangolin,ETH,B
